<a href="https://colab.research.google.com/github/keenanpepper/self-ablating-transformers/blob/main/Self_ablating_transformer_with_smooth_top_K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!git clone https://github.com/keenanpepper/self-ablating-transformers

Cloning into 'self-ablating-transformers'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 74 (delta 7), reused 14 (delta 4), pack-reused 42 (from 1)
Receiving objects: 100% (74/74), 57.09 MiB | 12.79 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [3]:
!git config --global user.email "keenanpepper@gmail.com"
!git config --global user.name "Keenan Pepper"

In [2]:
cd self-ablating-transformers/

/content/self-ablating-transformers


In [5]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.8/174.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import datasets
from tqdm.notebook import tqdm
import numpy as np
import wandb

/home/keenanpepper/.venvs/self-ablating-transformers/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from model.gpt_neo import GPTNeoWithSelfAblation
from model.config import GPTNeoWithSelfAblationConfig, TrainingConfig, WandBConfig

In [4]:
model = GPTNeoWithSelfAblation(GPTNeoWithSelfAblationConfig(hidden_size=128))

In [5]:
model

GPTNeoWithSelfAblation(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 128)
    (wpe): Embedding(2048, 128)
    (h): ModuleList(
      (0-7): 8 x GPTNeoBlockWithSelfAblation(
        (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): AttentionWithSelfAblation(
          (attention): ModuleDict(
            (k_proj): Linear(in_features=128, out_features=128, bias=False)
            (v_proj): Linear(in_features=128, out_features=128, bias=False)
            (q_proj): Linear(in_features=128, out_features=128, bias=False)
            (out_proj): Linear(in_features=128, out_features=128, bias=True)
          )
        )
        (ln_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): MLPWithSelfAblation(
          (c_fc): Linear(in_features=128, out_features=512, bias=True)
          (c_proj): Linear(in_features=512, out_features=128, bias=True)
          (act): NewGELUActivation()
        )
        (attention_ablation_head): Linear(i

In [6]:
model(torch.zeros((1,1), dtype=torch.long))

{'logits_clean': tensor([[[ 0.1357, -0.1861,  0.0825,  ..., -0.4153,  0.9947,  0.1479]]],
        grad_fn=<UnsafeViewBackward0>),
 'logits_ablated': tensor([[[-0.1530, -0.4126, -0.4074,  ..., -0.4040,  0.9786, -0.1074]]],
        grad_fn=<UnsafeViewBackward0>)}

In [7]:
ds = datasets.load_dataset("tdooms/TinyStories")

In [8]:
from utils import prepare_data

In [9]:
data, tokenizer = prepare_data(separator_token="<|endoftext|>")
print(f"Processed data shape: {data.shape}")
print(f"First 10 tokens: {data[:10]}")
print(f"Last 10 tokens: {data[-10:]}")

# Decode a small portion to verify
sample = data[:10000]
decoded = tokenizer.decode(sample.tolist())
print("\nSample decoded text:")
print(decoded[:5000] + "...")  # Print first 500 characters

train.bin already exists. Skipping processing.
Processed data shape: (396967656,)
First 10 tokens: [ 8888    11 19919   373   845  6568    13   679   373  1016]
Last 10 tokens: [  262 25103 12023   290  8359   257 12625  8073    13 50256]

Sample decoded text:
Today, Tommy was very excited. He was going flying with his mom and dad to a special place. There was a big green flag waving in the wind when they arrived. Tommy was so happy to see it. 

He hopped out of the car and ran up to the flag. He couldn't believe how big it was! He wanted to reach out and touch it. His mom said he could and Tommy smiled.

He waved his little arms, trying to make the flag move. But the wind was too strong. His dad said, "Let me help you, Tommy." He reached out and grabbed the green flag and waved it back and forth. 

Now it was Tommy's turn! He clapped his hands and laughed as he tried to move the flag. He waved it for a long time until he was too tired to do it anymore. The big green flag was so exciti

In [10]:
prepare_data(split="validation", output_file="validation.bin")

validation.bin already exists. Skipping processing.


(memmap([ 7454,  2402,   257, ..., 20567,    13, 50256], dtype=uint16),
 <Encoding 'gpt2'>)

In [11]:
class BatchGenerator:
    def __init__(self, data_file, block_size, batch_size, device):
        self.data_file = data_file
        self.block_size = block_size
        self.batch_size = batch_size
        self.device = torch.device(device)
        self.device_type = 'cuda' if 'cuda' in self.device.type else 'cpu'

    def get_batch(self, shifted=True):
        # We recreate np.memmap every batch to avoid a memory leak, as per
        # https://stackoverflow.com/questions/45132940/numpy-memmap-memory-usage-want-to-iterate-once/61472122#61472122
        data = np.memmap(self.data_file, dtype=np.uint16, mode='r')

        # Generate random starting indices
        ix = torch.randint(len(data) - self.block_size, (self.batch_size,))

        shift = 1 if shifted else 0
        # Create input and target tensors
        x = torch.stack([torch.from_numpy((data[i:i+self.block_size]).astype(np.int64)) for i in ix])
        y = torch.stack([torch.from_numpy((data[i+shift:i+shift+self.block_size]).astype(np.int64)) for i in ix])

        if self.device_type == 'cuda':
            # Pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
            x, y = x.pin_memory().to(self.device, non_blocking=True), y.pin_memory().to(self.device, non_blocking=True)
        else:
            x, y = x.to(self.device), y.to(self.device)

        return x, y

In [12]:
class LossEstimator:
    def __init__(self, model, train_batch_gen, val_batch_gen, eval_iters):
        self.model = model
        self.train_batch_gen = train_batch_gen
        self.val_batch_gen = val_batch_gen
        self.eval_iters = eval_iters

    @torch.no_grad()
    def estimate_loss(self):
        out = {}
        self.model.eval()
        for split, batch_gen in [('train', self.train_batch_gen), ('val', self.val_batch_gen)]:
            losses = torch.zeros(self.eval_iters)
            losses_clean = torch.zeros(self.eval_iters)
            losses_ablated = torch.zeros(self.eval_iters)
            reconstruction_losses = torch.zeros(self.eval_iters)
            for k in range(self.eval_iters):
                X, Y = batch_gen.get_batch()
                with torch.no_grad():
                    outputs = self.model(X, Y)
                    losses[k] = outputs['loss'].item()
                    losses_clean[k] = outputs['loss_clean'].item()
                    losses_ablated[k] = outputs['loss_ablated'].item()
                    reconstruction_losses[k] = outputs['reconstruction_loss'].item()

            out[split] = {
                'loss': losses.mean().item(),
                'loss_clean' : losses_clean.mean().item(),
                'loss_ablated' : losses_ablated.mean().item(),
                'reconstruction_loss' : reconstruction_losses.mean().item()
            }
        self.model.train()
        return out

In [13]:
def train_gptneo(model, config):

    # PLEASE update these values for each real training run you do - it will really help us keep track
    wandb_config = WandBConfig(model.config,
                               config,
                               dataset_name="TinyStories",
                               ablation_processing="soft-top-K-version-1",
                               reconstruction_loss="MSE")

#    wandb.init(project="gpt-neo-self-ablation", config=wandb_config)

    train_batch_gen = BatchGenerator(config.train_file, config.block_size, config.batch_size, config.device)
    val_batch_gen = BatchGenerator(config.val_file, config.block_size, config.batch_size, config.device)
    loss_estimator = LossEstimator(model, train_batch_gen, val_batch_gen, config.eval_iters)
    model.to(config.device)
    optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.num_batches)
    best_val_loss = float('inf')
    
    for iteration in tqdm(range(config.num_batches)):
        model.train()
        # Get batch
        x, y = train_batch_gen.get_batch()
        
        # Forward pass
        train_outputs = model(x, targets=y)
        loss = train_outputs['loss']
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        if config.max_grad_norm:
            nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
        optimizer.step()
        scheduler.step()

        # log progress every log_interval iterations
        if (iteration + 1) % config.log_interval == 0:
            stats = loss_estimator.estimate_loss()
            print(f"Iteration {iteration}: train loss {stats['train']['loss']:.4f}, val loss {stats['val']['loss']:.4f}")
#            wandb.log(stats | {"iteration": iteration, "current_learning_rate": optimizer.param_groups[0]['lr']})

            # Save best model
            if stats['val']['loss'] < best_val_loss:
                best_val_loss = stats['val']['loss']
                torch.save(model.state_dict(), config.save_path)
                print(f"New best model saved to {config.save_path}")
#                wandb.save(config.save_path) # Save the model to wandb
#                print(f"Model saved to wandb")

    print("Training completed!")
#    wandb.finish() # Finish the wandb run

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [15]:
from tqdm import tqdm

In [16]:
config = GPTNeoWithSelfAblationConfig(hidden_size=128, max_position_embeddings=256)
model = GPTNeoWithSelfAblation(config)

training_config = TrainingConfig(num_batches=1000, log_interval=100, batch_size=1)

train_gptneo(model, training_config)

 10%|▉         | 99/1000 [00:03<00:32, 27.73it/s]


KeyError: 'L_base'

In [114]:
config = GPTNeoWithSelfAblationConfig(hidden_size=128, max_position_embeddings=256)
model = GPTNeoWithSelfAblation(config)
model.load_state_dict(torch.load("best_model.pt"))
model.to(device)

<ipython-input-114-b14ef4e554fd>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pt"))


GPTNeoWithSelfAblation(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 128)
    (wpe): Embedding(256, 128)
    (h): ModuleList(
      (0-7): 8 x GPTNeoBlockWithSelfAblation(
        (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): AttentionWithSelfAblation(
          (attention): ModuleDict(
            (k_proj): Linear(in_features=128, out_features=128, bias=False)
            (v_proj): Linear(in_features=128, out_features=128, bias=False)
            (q_proj): Linear(in_features=128, out_features=128, bias=False)
            (out_proj): Linear(in_features=128, out_features=128, bias=True)
          )
        )
        (ln_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): MLPWithSelfAblation(
          (c_fc): Linear(in_features=128, out_features=512, bias=True)
          (c_proj): Linear(in_features=512, out_features=128, bias=True)
          (act): NewGELUActivation()
        )
      )
    )
    (ln_f): LayerNorm((128,), 

In [115]:
prompt_tlt = tokenizer.encode("One day, Tim and Lily went to the park. Tim found an interesting rock and gave it to")

In [116]:
ret_tlt = model(torch.tensor(prompt_tlt).unsqueeze(0).to(device))

In [117]:
ret2_tlt = model(torch.tensor(prompt_tlt).unsqueeze(0).to(device), attention_ablations=ret["attention_ablations"], neuron_ablations=ret["neuron_ablations"])

In [118]:
tokenizer.decode(torch.argmax(ret2_tlt["logits"],-1)[0,-1:].tolist())

' Lily'

In [119]:
prompt_tll = tokenizer.encode("One day, Tim and Lily went to the park. Lily found an interesting rock and gave it to")

In [120]:
ret_tll = model(torch.tensor(prompt_tll).unsqueeze(0).to(device))

In [126]:
ret2_tll = model(torch.tensor(prompt_tll).unsqueeze(0).to(device), attention_ablations=torch.heaviside(ret_tlt["attention_ablations"], torch.zeros([1],device=device)), neuron_ablations=torch.heaviside(ret_tll["neuron_ablations"], torch.zeros([1],device=device)))

In [127]:
tokenizer.decode(torch.argmax(ret2_tll["logits"],-1)[0,-1:].tolist())

' at'

In [130]:
import plotly.express as px

In [137]:
px.histogram(ret_tll["neuron_ablations"].flatten().detach().cpu(), log_y=True)

In [139]:
ret_tll["neuron_ablations"].max().item()

0.3707616627216339

In [142]:
from model.gpt_neo import soft_top_k